In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import OneClassSVM
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('df_features.csv')

In [4]:
def train_test_split(df, test_size=0.3):
    import random
    match_ids = df['matchId'].unique().tolist()
    train_size = int(len(match_ids) * (1 - test_size))
    train_match_ids = random.sample(match_ids, train_size)

    train = df[df['matchId'].isin(train_match_ids)]
    test = df[-df['matchId'].isin(train_match_ids)]

    return train, test

In [19]:
1-500000/len(df)

0.8866649787304166

In [20]:
train,test = train_test_split(df,0.9)

In [21]:
pd.options.display.max_columns = None
train.head()

,Unnamed: 0,Id,groupId,matchId,assists,killPlace,longestKill,revives,roadKills,teamKills,vehicleDestroys,weaponsAcquired,winPlacePerc,totalDistance,headshotPerc,killsNorm,damageDealtNorm,DBNOsNorm,killstreakOverKills,killsPerDistance,weaponPerDistance,teamContribution,healsAndBoosts,healsAndBoostsPerWalkDistance,healsAndBoostsPerKill
6,6,95959be0e21ca3,2c485a1ad3d0f1,a8274e903927a2,0,96,0.000,0,0,0,0,1,0.0000,13.50,0.0,0.0,0.0000,0.0,0.0,0.000000e+00,1.000000e-31,-0.607622,0,0.000000e+00,0.000000e+00
9,9,e5bb5a43587253,759bb6f7514fd2,3d3031c795305b,0,74,0.000,0,0,0,0,1,0.2143,65.67,0.0,0.0,38.3881,0.0,0.0,0.000000e+00,1.000000e-31,-0.607622,0,0.000000e+00,0.000000e+00
20,20,02ace8c6e58461,a4bc548028f800,80f2b8448e474b,0,18,7.438,1,0,0,0,5,0.6383,4528.00,0.5,2.1,282.5550,0.0,0.5,2.100000e-31,5.000000e-31,1.516244,12,1.200000e-30,1.200000e-30
31,31,4d16222292e9c8,539c5ba5a4ca96,9d090204befa18,0,80,0.000,0,0,0,0,2,0.0667,93.11,0.0,0.0,0.0000,0.0,0.0,0.000000e+00,2.000000e-31,-0.607622,0,0.000000e+00,0.000000e+00
32,32,e050d1f179d289,9e9fc09bcdab3f,bf3065e01dd3f9,0,61,0.000,0,0,1,0,7,0.5000,2394.00,0.0,0.0,0.0000,0.0,0.0,0.000000e+00,7.000000e-31,-6.628916,4,4.000000e-31,4.000000e-31


In [22]:
len(train)

440637

In [23]:
outliers_fraction = 0.1
data = train[['roadKills','killsNorm','weaponPerDistance','healsAndBoostsPerKill','headshotPerc']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
# train one class SVM 
model =  OneClassSVM(nu=0.95 * outliers_fraction,verbose=False) #nu=0.95 * outliers_fraction  + 0.05
data = pd.DataFrame(np_scaled)
model.fit(data)
# add the data to the main  
train['anomaly'] = model.predict(data)
train['anomaly'] = train['anomaly'].map( {1: 0, -1: 1} )
print(train['anomaly'].value_counts())

0    396229
1     44408
Name: anomaly, dtype: int64


In [34]:
len(model.predict(data))

4385

In [38]:
train.groupby('anomaly').max()

,Unnamed: 0,Id,groupId,matchId,assists,killPlace,longestKill,revives,roadKills,teamKills,vehicleDestroys,weaponsAcquired,winPlacePerc,totalDistance,headshotPerc,killsNorm,damageDealtNorm,DBNOsNorm,killstreakOverKills,killsPerDistance,weaponPerDistance,teamContribution,healsAndBoosts,healsAndBoostsPerWalkDistance,healsAndBoostsPerKill,anomaly26
anomaly,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4445848,ffea11134b03c2,ffed4052ea8616,ff7fe5c76f1e44,5,99,547.9,4,0,3,3,11,1.0,25748.5,1.0,7.49,1080.696,8.32,1.0,7.490000e-31,1.100000e-30,13.147213,17,1.700000e-30,1.700000e-30,0.0
1,4438319,ff2ecbde597f8c,ffed4052ea8616,ff7fe5c76f1e44,5,83,438.0,6,3,1,1,16,1.0,11562.0,1.0,13.65,1772.630,14.98,1.0,1.365000e-30,1.600000e-30,13.888702,32,3.200000e-30,3.200000e-30,NaN


NameError: name 'data_sub' is not defined